<a href="https://colab.research.google.com/github/EverlynAsiko/Neural_Machine_Translation_for_African_Languages/blob/main/KinyarwandaLuhya_Multilingual_NMT_results1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual neural machine translation.

For this case, we shall to a many-to-one translation:
{Kinyarwanda, Luhya} to English. 

In [ ]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Importing needed libraries for preprocessing and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#@title Default title text
# Install Pytorch with GPU support v1.8.0.
! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.


In [ ]:
# Filtering warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shared drives/NMT_for_African_Language")

In [ ]:
# Setting source and target languages
source_language = "en"
target_language = "rw_lh"

os.environ["src"] = source_language 
os.environ["tgt"] = target_language

## Data preprocessing

In [ ]:
! head Kinyarwanda/train.*
! head Kinyarwanda/dev.*

==> Kinyarwanda/train.bpe.en <==
R@@ ight after his bapt@@ ism , he “ went off into Ar@@ ab@@ ia ” ​ — e@@ ither the S@@ y@@ ri@@ an D@@ es@@ ert or pos@@ sib@@ ly some qu@@ i@@ et place on the Ar@@ ab@@ ian P@@ en@@ ins@@ ul@@ a that was conduc@@ ive to med@@ it@@ ation .
You will see the time when God br@@ ings righteous rule to all the earth , und@@ o@@ ing the d@@ am@@ age and inj@@ ust@@ ice brought by human rul@@ er@@ ship .
Let us consider f@@ ive reas@@ ons why we should want to follow the Christ .
Even in the Bible , the id@@ ea of pers@@ u@@ as@@ ion som@@ et@@ imes has n@@ eg@@ ative con@@ no@@ t@@ ations , den@@ ot@@ ing a cor@@ rup@@ ting or a lead@@ ing as@@ tr@@ ay .
For God’s servants to be deliv@@ ered , Satan and his ent@@ ire world@@ wide system of things need to be rem@@ ov@@ ed .
I had never heard that name used in my ch@@ urch .
S@@ imp@@ ly having authority or a wid@@ er name recogn@@ ition is not the important thing .
M@@ ost people do not believe in the spir@@ 

In [ ]:
! head Luhyia/train.*
! head Luhyia/dev.*

==> Luhyia/train.bpe.en <==
 T@@ hat day was the P@@ re@@ par@@ ation, and the Sab@@ b@@ ath drew ne@@ ar@@ .
 Behold, I am coming qui@@ ck@@ l@@ y@@ ! H@@ old fast what you ha@@ ve, that no one may take your crow@@ n.
 The next day, because he wan@@ ted to know for certain why he was acc@@ us@@ ed by the Jews, he r@@ ele@@ as@@ ed him from his bond@@ s, and commanded the chief priests and all their coun@@ ci@@ l to appear@@ , and brought Paul down and set him before them. 

 This He said, sig@@ ni@@ f@@ ying by what death He would di@@ e.
 Then they said to the wom@@ an, “@@ Now we believ@@ e, not because of what you said, for we our@@ selves have heard Him and we know that this is indeed the Christ, the Sa@@ vi@@ or of the worl@@ d.”
 But rej@@ ect prof@@ ane and old wi@@ ves@@ ’ f@@ ab@@ les, and ex@@ er@@ c@@ ise your@@ self toward god@@ lin@@ es@@ s.
 It is written in the prophe@@ ts, ‘@@ And they shall all be taught by God@@ .’ Therefore everyone who has heard and lear@@ ned from

In [ ]:
pre = '/content/gdrive/Shared drives/NMT_for_African_Language/'
# Train data source
filenames = [pre+'Kinyarwanda/train.en',pre+'Luhyia/train.en']

# Train data target
filenames2 = [pre+'Kinyarwanda/train.rw',pre+'Luhyia/train.lh']

# Dev data source
file1 = [pre+'Kinyarwanda/dev.en',pre+'Luhyia/dev.en']

# Dev data target
file2 = [pre+'Kinyarwanda/dev.rw',pre+'Luhyia/dev.lh']

In [ ]:
# Changing to Multilingual directory
os.chdir("/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3")

In [ ]:
# Procedure to create concatenated files
def create_file(x,filename):
  # Open filename in write mode
  with open(filename, 'w') as outfile:
      for names in x:
          # Open each file in read mode
          with open(names) as infile:
              # read the data and write it in file3
              outfile.write(infile.read())
          outfile.write("\n")

In [ ]:
create_file(filenames,'train.en')

In [ ]:
create_file(filenames2,'train.rw_lh')

In [ ]:
create_file(file1,'dev.en')

In [ ]:
create_file(file2,'dev.rw_lh')

### BPE codes

In [ ]:
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Processing /content/gdrive/Shareddrives/NMT_for_African_Language/Multilingual3/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 15.3 MB 94 kB/s 
     |████████████████████████████████| 7.1 MB 27.6 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 636 kB 40.5 MB/s 
     |████████████████████████████████| 375 kB 23.3 MB/s 
     |████████████████████████████████| 105 kB 50.3 MB/s 
     |████████████████████████████████| 231 kB 48.7 MB/s 
     |████████████████████████████████| 743 kB 35.3 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 


In [ ]:
# Apply BPE splits to the development and test data.
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py train.bpe.$src train.bpe.$tgt --output_path vocab.txt

In [ ]:
# Applying BPE to tests
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test1.$src > test.bpe.en1
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test1.lh > test.bpe.lh

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test2.$src > test.bpe.en2
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test2.rw > test.bpe.rw

In [ ]:
# Some output
! echo "BPE Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 vocab.txt

BPE Sentences
N@@ asi , n@@ ir@@ ee@@ ba end@@ i , ‘ N@@ iwe w@@ ina , O@@ mw@@ ami ? ’ O@@ mw@@ o@@ yo ok@@ wo , n@@ ik@@ umb@@ ool@@ ela kuri , ‘ N@@ is@@ ie Yesu o@@ wa N@@ azar@@ eti ow@@ os@@ a@@ and@@ inj@@ ia . ’
sh@@ ich@@ ila , om@@ uk@@ ha@@ an@@ awe om@@ ut@@ el@@ wa , ow@@ em@@ iy@@ ika ek@@ hum@@ i n@@ ach@@ ib@@ ili y@@ ali n@@ any@@ ir@@ anga . N@@ e ol@@ wa y@@ ali nat@@ s@@ it@@ s@@ anga , aband@@ u , bam@@ wib@@ um@@ bak@@ h@@ wo okh@@ ur@@ ula mut@@ s@@ imb@@ eka t@@ si@@ o@@ si .
N@@ e ol@@ wa kab@@ is@@ ib@@ wa mbu kh@@ uk@@ ho@@ y@@ ile okh@@ uts@@ i@@ ila , m@@ um@@ e@@ eli okh@@ u@@ ula It@@ al@@ ia , bah@@ a@@ ana Paul@@ o nende abab@@ o@@ he , bandi k@@ hum@@ us@@ inj@@ il@@ ili w@@ el@@ ihe J@@ ul@@ i@@ asi ow@@ e@@ ing@@ '@@ anda e@@ ya , esh@@ ir@@ oma ey@@ il@@ angwa mbu , “ I@@ ng@@ '@@ anda ey@@ il@@ ind@@ anga , O@@ mur@@ uc@@ h@@ i . ”
O@@ l@@ uny@@ um@@ akh@@ wo , abak@@ u@@ uka b@@ ef@@ we , abab@@ uk@@ ula l@@ ih@@ e@@ ema el@@ o okh@@ ur@@ ula kh@@

In [ ]:
! tail train.*
! tail dev.*

==> train.bpe.en <==
But if anyone lov@@ es God , this one is known by Him .
And the second is like it : ‘ You shall love your neigh@@ b@@ or as yourself . ’
until the day in which He was taken up , after He through the H@@ o@@ ly Sp@@ ir@@ it had given command@@ ments to the apostles whom He had cho@@ s@@ en ,
For what if some did not believe ? W@@ ill their un@@ beli@@ ef make the faith@@ ful@@ ness of God without ef@@ fect ?
And when you go into a hous@@ eho@@ ld , gre@@ et it .
And a very great mul@@ t@@ itude sp@@ read their clo@@ th@@ es on the ro@@ ad ; others c@@ ut down br@@ an@@ ch@@ es from the tre@@ es and sp@@ read them on the ro@@ ad .
And we heard this vo@@ ice which came from heaven when we were with Him on the holy m@@ ount@@ ain .
O@@ r those e@@ igh@@ te@@ en on whom the tower in S@@ il@@ o@@ am f@@ ell and k@@ ill@@ ed them , do you think that they were wor@@ se sin@@ n@@ ers than all other men who dw@@ el@@ t in Jerusalem ?
For I be@@ ar him witness that he has a g

## Modelling

### Baseline

In [ ]:
#@title
name = '%s%s' % (target_language, source_language)

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 1000
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 5000         # TODO: Set to at least once per epoch.
    logging_freq: 200
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer"
    overwrite: True 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path="/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3", source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
# Train the model
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-07-13 10:50:31,579 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-13 10:50:31,657 - INFO - joeynmt.data - Loading training data...
2021-07-13 10:50:42,167 - INFO - joeynmt.data - Building vocabulary...
2021-07-13 10:50:42,507 - INFO - joeynmt.data - Loading dev data...
2021-07-13 10:50:42,605 - INFO - joeynmt.data - Loading test data...
2021-07-13 10:50:43,790 - INFO - joeynmt.data - Data loaded.
2021-07-13 10:50:43,790 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-13 10:50:44,196 - INFO - joeynmt.model - Enc-dec model built.
2021-07-13 10:50:44.460578: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 10:50:46,651 - INFO - joeynmt.training - Total params: 12177920
2021-07-13 10:50:55,312 - INFO - joeynmt.helpers - cfg.name                           : rw_lhen_reverse_transformer
2021-07-13 10:50:55,312 - INFO - joeynmt.helpers - cfg.data.src                       

16 epochs ran

In [ ]:
# Reloading configuration file
ckpt_number = 85000
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/models/rw_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/rw_lhen_reverse_transformer"', f'model_dir: "models/rw_lhen_reverse_transformer_continued"').replace(
        f'epochs: 30', f'epochs: 14')
        
with open("joeynmt/configs/transformer_{name}_reload.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
!cat "joeynmt/configs/transformer_rw_lhen_reload.yaml"


name: "rw_lhen_reverse_transformer"

data:
    src: "rw_lh"
    trg: "en"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer/85000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # T

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_rw_lhen_reload.yaml

2021-07-14 13:08:58,914 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 13:08:58,982 - INFO - joeynmt.data - Loading training data...
2021-07-14 13:09:09,121 - INFO - joeynmt.data - Building vocabulary...
2021-07-14 13:09:09,726 - INFO - joeynmt.data - Loading dev data...
2021-07-14 13:09:10,601 - INFO - joeynmt.data - Loading test data...
2021-07-14 13:09:11,368 - INFO - joeynmt.data - Data loaded.
2021-07-14 13:09:11,368 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 13:09:11,587 - INFO - joeynmt.model - Enc-dec model built.
2021-07-14 13:09:11.842929: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-14 13:09:13,491 - INFO - joeynmt.training - Total params: 12177920
2021-07-14 13:09:23,401 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer/85000.ckpt
2021-07-

14 epochs complete

16+14 = 30 epochs complete

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/rw_lhen_reverse_transformer/validations.txt"

Steps: 5000	Loss: 209669.92188	PPL: 28.52874	bleu: 3.84511	LR: 0.00030000	*
Steps: 10000	Loss: 186578.73438	PPL: 19.72471	bleu: 6.93421	LR: 0.00030000	*
Steps: 15000	Loss: 176237.12500	PPL: 16.71981	bleu: 8.03345	LR: 0.00030000	*
Steps: 20000	Loss: 167712.25000	PPL: 14.59021	bleu: 9.32608	LR: 0.00030000	*
Steps: 25000	Loss: 162152.32812	PPL: 13.34969	bleu: 9.93043	LR: 0.00030000	*
Steps: 30000	Loss: 157637.42188	PPL: 12.42036	bleu: 11.15394	LR: 0.00030000	*
Steps: 35000	Loss: 155305.07812	PPL: 11.96591	bleu: 11.75460	LR: 0.00030000	*
Steps: 40000	Loss: 152631.00000	PPL: 11.46530	bleu: 11.44089	LR: 0.00030000	*
Steps: 45000	Loss: 150888.71875	PPL: 11.15045	bleu: 12.76499	LR: 0.00030000	*
Steps: 50000	Loss: 149148.78125	PPL: 10.84466	bleu: 12.32629	LR: 0.00030000	*
Steps: 55000	Loss: 147339.18750	PPL: 10.53552	bleu: 13.28245	LR: 0.00030000	*
Steps: 60000	Loss: 145776.71875	PPL: 10.27569	bleu: 12.85105	LR: 0.00030000	*
Steps: 65000	Loss: 144349.95312	PPL: 10.04403	bleu: 13.63222	LR: 0.000

In [ ]:
! cat "joeynmt/models/rw_lhen_reverse_transformer_continued/validations.txt"

Steps: 90000	Loss: 139744.14062	PPL: 9.33125	bleu: 14.27456	LR: 0.00030000	*
Steps: 95000	Loss: 139600.45312	PPL: 9.30985	bleu: 14.60423	LR: 0.00030000	*
Steps: 100000	Loss: 138584.57812	PPL: 9.15992	bleu: 14.38571	LR: 0.00030000	*
Steps: 105000	Loss: 139485.21875	PPL: 9.29272	bleu: 15.10734	LR: 0.00030000	
Steps: 110000	Loss: 137181.06250	PPL: 8.95674	bleu: 15.45227	LR: 0.00030000	*
Steps: 115000	Loss: 138259.09375	PPL: 9.11239	bleu: 15.49028	LR: 0.00030000	
Steps: 120000	Loss: 136506.10938	PPL: 8.86064	bleu: 15.66517	LR: 0.00030000	*
Steps: 125000	Loss: 136562.43750	PPL: 8.86862	bleu: 15.48623	LR: 0.00030000	
Steps: 130000	Loss: 136139.18750	PPL: 8.80884	bleu: 15.73478	LR: 0.00030000	*
Steps: 135000	Loss: 135421.07812	PPL: 8.70832	bleu: 15.83303	LR: 0.00030000	*
Steps: 140000	Loss: 135106.35938	PPL: 8.66463	bleu: 16.09501	LR: 0.00030000	*
Steps: 145000	Loss: 137186.35938	PPL: 8.95750	bleu: 15.89274	LR: 0.00030000	
Steps: 150000	Loss: 134481.78125	PPL: 8.57857	bleu: 16.04871	LR: 0.000

In [ ]:
!cd joeynmt; python -m joeynmt test 'models/rw_lhen_reverse_transformer_continued/config.yaml'

2021-07-14 16:37:51,568 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 16:37:51,573 - INFO - joeynmt.data - Building vocabulary...
2021-07-14 16:37:51,850 - INFO - joeynmt.data - Loading dev data...
2021-07-14 16:37:51,878 - INFO - joeynmt.data - Loading test data...
2021-07-14 16:37:51,894 - INFO - joeynmt.data - Data loaded.
2021-07-14 16:37:51,925 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 5000 (with beam_size)
2021-07-14 16:37:55,578 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 16:37:55,773 - INFO - joeynmt.model - Enc-dec model built.
2021-07-14 16:37:55,842 - INFO - joeynmt.prediction - Decoding on dev set (/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/dev.bpe.en)...
2021-07-14 16:39:14,072 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-07-14 16:39:14,073 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, w

In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/rw_lhen_reverse_transformer_continued/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe.lh" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/translation.bpe.lh_en"

2021-07-14 16:40:20,238 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 16:40:24,207 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 16:40:24,403 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!cat "translation.bpe.lh_en" | sacrebleu "test1.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 9.1 38.4/13.1/5.3/2.6 (BP = 1.000 ratio = 1.018 hyp_len = 26521 ref_len = 26044)


In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/rw_lhen_reverse_transformer_continued/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe.rw" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/translation.bpe.rw_en"

2021-07-14 16:41:26,994 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 16:41:30,972 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 16:41:31,169 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
#@title
def empty_counter(x):
  # Opening a file
  infile = open(x,"r")
  empty = []
  
  for i,line in enumerate(infile):
    if not line.strip(): 
      empty.append(i)

  return empty

In [ ]:
#@title
# Reference: https://thispointer.com/python-how-to-delete-specific-lines-in-a-file-in-a-memory-efficient-way/
def delete_multiple_lines(original_file, line_numbers):
    """In a file, delete the lines at line number in given list"""
    is_skipped = False
    counter = 0
    # Create name of dummy / temporary file
    dummy_file = original_file + '.bak'
    # Open original file in read only mode and dummy file in write mode
    with open(original_file, 'r') as read_obj, open(dummy_file, 'w') as write_obj:
        # Line by line copy data from original file to dummy file
        for line in read_obj:
            # If current line number exist in list then skip copying that line
            if counter not in line_numbers:
                write_obj.write(line)
            else:
                is_skipped = True
            counter += 1
    # If any line is skipped then rename dummy file as original file
    if is_skipped:
        os.remove(original_file)
        os.rename(dummy_file, original_file)
    else:
        os.remove(dummy_file)

In [ ]:
delete_multiple_lines("test2.en",empty_counter("test2.rw"))

In [ ]:
!cat "translation.bpe.rw_en" | sacrebleu "test2.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 37.7 67.9/46.2/35.2/27.9 (BP = 0.901 ratio = 0.906 hyp_len = 38435 ref_len = 42439)


Getting to 60 epochs

In [ ]:
# Reloading configuration file
ckpt_number = 160000
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/models/rw_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/rw_lhen_reverse_transformer"', f'model_dir: "models/rw_lhen_reverse_transformer_continued2"')
        
with open("joeynmt/configs/transformer_{name}_reload2.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_rw_lhen_reload2.yaml

2021-07-22 09:24:10,389 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-22 09:24:10,460 - INFO - joeynmt.data - Loading training data...
2021-07-22 09:24:22,130 - INFO - joeynmt.data - Building vocabulary...
2021-07-22 09:24:22,763 - INFO - joeynmt.data - Loading dev data...
2021-07-22 09:24:23,383 - INFO - joeynmt.data - Loading test data...
2021-07-22 09:24:24,078 - INFO - joeynmt.data - Data loaded.
2021-07-22 09:24:24,078 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-22 09:24:24,485 - INFO - joeynmt.model - Enc-dec model built.
2021-07-22 09:24:24.741675: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-22 09:24:26,889 - INFO - joeynmt.training - Total params: 12177920
2021-07-22 09:24:30,401 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued/160000.ck

In [ ]:
# Reloading configuration file
ckpt_number = 195000
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/models/rw_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued2/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/rw_lhen_reverse_transformer"', f'model_dir: "models/rw_lhen_reverse_transformer_continued3"').replace(
            f'epochs: 30', f'epochs: 22')
        
with open("joeynmt/configs/transformer_{name}_reload3.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
!cat "joeynmt/configs/transformer_{name}_reload3.yaml"


name: "rw_lhen_reverse_transformer"

data:
    src: "rw_lh"
    trg: "en"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued2/195000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"  

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_rw_lhen_reload3.yaml

2021-07-27 07:15:35,664 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-27 07:15:35,743 - INFO - joeynmt.data - Loading training data...
2021-07-27 07:15:48,636 - INFO - joeynmt.data - Building vocabulary...
2021-07-27 07:15:49,505 - INFO - joeynmt.data - Loading dev data...
2021-07-27 07:15:51,070 - INFO - joeynmt.data - Loading test data...
2021-07-27 07:15:52,593 - INFO - joeynmt.data - Data loaded.
2021-07-27 07:15:52,593 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-27 07:15:53,025 - INFO - joeynmt.model - Enc-dec model built.
2021-07-27 07:15:53.282177: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-27 07:15:55,390 - INFO - joeynmt.training - Total params: 12177920
2021-07-27 07:16:04,059 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued2/195000.c

In [ ]:
# Reloading configuration file
ckpt_number = 280000
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/models/rw_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued3/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/rw_lhen_reverse_transformer"', f'model_dir: "models/rw_lhen_reverse_transformer_continued4"').replace(
            f'epochs: 30', f'epochs: 6')
        
with open("joeynmt/configs/transformer_{name}_reload4.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
!cat "joeynmt/configs/transformer_{name}_reload4.yaml"


name: "rw_lhen_reverse_transformer"

data:
    src: "rw_lh"
    trg: "en"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued3/280000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"  

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_rw_lhen_reload4.yaml

2021-07-28 06:51:52,562 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-28 06:51:52,651 - INFO - joeynmt.data - Loading training data...
2021-07-28 06:52:05,063 - INFO - joeynmt.data - Building vocabulary...
2021-07-28 06:52:05,912 - INFO - joeynmt.data - Loading dev data...
2021-07-28 06:52:06,937 - INFO - joeynmt.data - Loading test data...
2021-07-28 06:52:08,269 - INFO - joeynmt.data - Data loaded.
2021-07-28 06:52:08,270 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-28 06:52:08,697 - INFO - joeynmt.model - Enc-dec model built.
2021-07-28 06:52:08.959827: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-28 06:52:11,276 - INFO - joeynmt.training - Total params: 12177920
2021-07-28 06:52:20,009 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/joeynmt/models/rw_lhen_reverse_transformer_continued3/280000.c

In [ ]:
!pwd

/content/gdrive/Shareddrives/NMT_for_African_Language/Multilingual3


In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/rw_lhen_reverse_transformer_continued4/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe.lh" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/translation1.bpe.lh_en"

2021-07-28 10:10:32,093 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-28 10:10:35,055 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-28 10:10:35,331 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!cat "translation1.bpe.lh_en" | sacrebleu "test1.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 10.7 42.7/16.1/7.1/3.6 (BP = 0.931 ratio = 0.933 hyp_len = 24311 ref_len = 26044)


In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/rw_lhen_reverse_transformer_continued4/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/test.bpe.rw" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual3/translation1.bpe.rw_en"

2021-07-28 10:38:30,969 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-28 10:38:33,971 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-28 10:38:34,243 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!cat "translation1.bpe.rw_en" | sacrebleu "test2.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 38.3 68.8/47.1/36.0/28.7 (BP = 0.894 ratio = 0.899 hyp_len = 38159 ref_len = 42439)
